In [28]:
import os
import json

# Define the path to your data
data_dir = os.path.join('problems', 'problems_pivot_rules_two_phases')

# Define pivot rules
pivot_rules = ['dantzig', 'random', 'steepest_edge']

# Example sizes (replace with your actual tuples for balanced_sizes, wide_sizes, and tall_sizes)
balanced_sizes = [(i, i) for i in range(5, 101, 5)]
wide_sizes = [(i, 2*i) for i in range(5, 51, 5)]
tall_sizes = [(2*i, i) for i in range(5, 51, 5)]
root_folder = os.path.join('problems', 'problems_pivot_rules_one_phase_only')

# Function to read and return the data from json files
def load_data(size_folder):
    with open(size_folder, 'r') as f:
        data = json.load(f)
    return data

# Function to calculate relative performance based on total time
def compute_relative_performance(data, baseline_rule='steepest_edge'):
    relative_performance = {}
    
    # Get the total_time for the baseline rule
    time_data = data['total_time']
    baseline_time = time_data[baseline_rule]
    
    for rule in pivot_rules:
        if rule != baseline_rule:
            # Relative time performance (relative to baseline rule)
            relative_time = time_data[rule] / baseline_time
            relative_performance[rule] = relative_time
    relative_performance[baseline_rule] = 1
    return relative_performance

# Function to compute average relative performance for the last two sizes of each category
def compute_avg_relative_performance(sizes_list, _type, baseline_rule='steepest_edge'):
    avg_relative_performance = {rule: 0 for rule in pivot_rules}  # Only for time
    
    for size in sizes_list:  # Only the last two sizes
        size_folder = os.path.join(data_dir, _type, f"{size[0]}x{size[1]}")
        data = load_data(os.path.join(size_folder, 'averages.json'))
        
        relative_performance = compute_relative_performance(data)
        for rule in pivot_rules:
            avg_relative_performance[rule] += relative_performance[rule]
    
    # Compute the averages
    for rule in pivot_rules:
        avg_relative_performance[rule] /= len(sizes_list)  # Average time relative performance
    
    return avg_relative_performance

# Now, for each category, calculate the average relative performance for the last two sizes
avg_balanced_performance = compute_avg_relative_performance(balanced_sizes, 'balanced_problems')
avg_wide_performance = compute_avg_relative_performance(wide_sizes, 'wide_problems')
avg_tall_performance = compute_avg_relative_performance(tall_sizes, 'tall_problems')

# Display the results
def print_avg_performance(category, avg_performance):
    print(f"\nAverage Relative Time Performance for {category}:")
    print(f"{'Pivot Rule':<15} {'Avg Time'}")
    for rule in pivot_rules:
        print(f"{rule:<15} {avg_performance[rule]:<10.2f}")

# Print the results for each category
print_avg_performance("Balanced", avg_balanced_performance)
print_avg_performance("Wide", avg_wide_performance)
print_avg_performance("Tall", avg_tall_performance)


Average Relative Time Performance for Balanced:
Pivot Rule      Avg Time
dantzig         2.02      
random          3.26      
steepest_edge   1.00      

Average Relative Time Performance for Wide:
Pivot Rule      Avg Time
dantzig         1.83      
random          3.08      
steepest_edge   1.00      

Average Relative Time Performance for Tall:
Pivot Rule      Avg Time
dantzig         1.30      
random          2.49      
steepest_edge   1.00      


In [ ]:
import os
import json

# Define pivot rules and categories
pivot_rules = ["dantzig", "steepest_edge", "random"]  # excluding "bland"
categories = ["balanced_problems", "tall_problems", "wide_problems"]
problem_sets = ["one_phase_only", "two_phases"]

# Function to load data from JSON files
def load_json_file(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            return json.load(f)
    else:
        print(f"File not found: {file_path}")
        return {}

# Storage for results
results = {category: {problem_set: {rule: [] for rule in pivot_rules} for problem_set in problem_sets} for category in categories}

# Iterate over each category (balanced, tall, wide)
for category in categories:
    # For one-phase and two-phase problem sets
    for problem_set in problem_sets:
        base_path = os.path.join("problems", f"problems_pivot_rules_{problem_set}", category)
        
        # Iterate over the size folders for each problem set
        for size_folder in os.listdir(base_path):
            size_folder_path = os.path.join(base_path, size_folder)
            
            if problem_set == "one_phase_only":
                # Load averages.json for one-phase problems
                averages_file = os.path.join(size_folder_path, "averages.json")
                data = load_json_file(averages_file)
                
                # Calculate the average times for each pivot rule
                for rule in pivot_rules:
                    if rule in data["total_time"]:
                        results[category][problem_set][rule].append(data["total_time"][rule])

            elif problem_set == "two_phases":
                # Load averages.json for two-phase problems (overall case)
                averages_file = os.path.join(size_folder_path, "averages.json")
                data = load_json_file(averages_file)
                
                # Calculate the average times for each pivot rule in the general two-phase case
                for rule in pivot_rules:
                    if rule in data["total_time"]:
                        results[category][problem_set][rule].append(data["total_time"][rule])
                
                # For Infeasible, Optimal, Unbounded, load respective files
                for status in ["Optimal", "Infeasible", "Unbounded"]:
                    status_file = os.path.join(size_folder_path, f"averages_{status}.json")
                    status_data = load_json_file(status_file)
                    
                    # Store the times for each status
                    for rule in pivot_rules:
                        print(status_data["total_time"])
                        if rule in status_data["total_time"]:
                            results[category][problem_set][rule].append(status_data["total_time"][rule])

# Now, compute the averages for each category, problem set, and pivot rule
final_results = {}

# Iterate through the results and calculate the average for each category and pivot rule
for category, problem_data in results.items():
    final_results[category] = {}
    
    for problem_set, rule_data in problem_data.items():
        final_results[category][problem_set] = {}
        
        for rule, time_data in rule_data.items():
            # Calculate the average for each pivot rule, in each problem set
            if len(time_data) > 0:
                final_results[category][problem_set][rule] = sum(time_data) / len(time_data)
            else:
                final_results[category][problem_set][rule] = 0

# Now, final_results contains the average total times for each problem size, for each pivot rule, for one-phase and two-phase problems
# Print or store the results as needed
print(json.dumps(final_results, indent=4))


{'total_time': {'dantzig': 53015.161532622114, 'random': 77790.8789561345, 'steepest_edge': 15740.769386291504}, 'num_pivot_steps_first_phase': {'dantzig': 320.0, 'random': 643.0769230769231, 'steepest_edge': 133.6153846153846}, 'num_pivot_steps_second_phase': {'dantzig': 46.30769230769231, 'random': 88.53846153846153, 'steepest_edge': 26.0}, 'num_pivot_steps_total': {'dantzig': 366.3076923076923, 'random': 731.6153846153846, 'steepest_edge': 159.6153846153846}}
{'total_time': {'dantzig': 53015.161532622114, 'random': 77790.8789561345, 'steepest_edge': 15740.769386291504}, 'num_pivot_steps_first_phase': {'dantzig': 320.0, 'random': 643.0769230769231, 'steepest_edge': 133.6153846153846}, 'num_pivot_steps_second_phase': {'dantzig': 46.30769230769231, 'random': 88.53846153846153, 'steepest_edge': 26.0}, 'num_pivot_steps_total': {'dantzig': 366.3076923076923, 'random': 731.6153846153846, 'steepest_edge': 159.6153846153846}}
{'total_time': {'dantzig': 53015.161532622114, 'random': 77790.878

KeyError: 'total_time'